In [1]:
#Libraries to import
import numpy as np 
import pandas as pd
import os
from PIL import Image
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa
from sklearn.utils import shuffle

In [2]:
#PARAMETERS
batch_size = 32
num_classes = 3
epochs = 20
train_size = 0.8 #80-20 train validation split
#File source using kaggle format
train_source = "../input/deforestation/train.csv"
test_source = "../input/deforestation/test.csv"
dataset_source = "../input/deforestation/train_test_data/"
#Some base model sources for transfer learning (We will use inception_V3)
euro_res = "https://tfhub.dev/google/remote_sensing/eurosat-resnet50/1"
resisc_net = "https://tfhub.dev/google/remote_sensing/resisc45-resnet50/1"
uc_merced_net = "https://tfhub.dev/google/remote_sensing/uc_merced-resnet50/1"
inception_V3 = "https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/5"
res_net = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/5"
eff_net_B3 = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_b3/feature_vector/2"
res_net_101 = "https://tfhub.dev/google/imagenet/resnet_v2_101/feature_vector/5"


In [3]:
#Returns the dataset of images as a batch of numpy arrays
def get_np_data(source, dataset_source, get_labels=False):
    df = pd.read_csv(source)
    total_data = []
    for image_path in df["example_path"]:
        image_data = np.asarray(Image.open(dataset_source+image_path))
        total_data.append(image_data)
    
    total_data = np.asarray(total_data)/255
    if(get_labels):
        total_labels = np.asarray(df["label"])
        return total_data, total_labels
    else:
        return total_data

In [4]:
def get_tf_dataset(total_data, total_labels=None, get_labels=False, augment=False):
    if(get_labels):
        ds = tf.data.Dataset.from_tensor_slices((total_data,total_labels))
        
        if(augment):
            #Add random flips for data augmentation for the training data
            trainAug = tf.keras.Sequential([
                tf.keras.layers.RandomFlip("horizontal_and_vertical"),
                ])
            #Cache and prefetch the datasets
            ds = (
                ds
                .cache()
                .batch(batch_size)
                .map(lambda x, y: (trainAug(x), y),
                num_parallel_calls=tf.data.AUTOTUNE)
                .prefetch(tf.data.AUTOTUNE)
            )
            return ds
        
        ds = ds.cache().batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
        return ds
    
    else:
        ds = tf.data.Dataset.from_tensor_slices((total_data))
        ds = ds.cache().batch(batch_size).prefetch(buffer_size=tf.data.AUTOTUNE)
        return ds

In [5]:
#80% Split of train data into training and validation
total_train_data, total_train_labels = get_np_data(train_source, dataset_source, get_labels=True)
input_shape = total_train_data.shape[1:] #Get the shape of the images
#Split the data
train_data, val_data, train_labels, val_labels = train_test_split(total_train_data, total_train_labels, train_size=train_size)
del total_train_data, total_train_labels #delete to save memory

#Turn into tensorflow dataset
train_ds = get_tf_dataset(train_data, train_labels, get_labels=True, augment = True)
del train_data, train_labels
val_ds = get_tf_dataset(val_data, val_labels, get_labels=True, augment = False)
del val_data, val_labels


2022-11-19 21:55:08.111324: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-19 21:55:08.210591: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-19 21:55:08.211633: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-19 21:55:08.215245: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [6]:
def build_model(base_model, input_shape, num_classes):

    
    inputs = tf.keras.Input(shape = input_shape)

    x = base_model(inputs, training=False)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(100, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(100, activation="relu")(x)

    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)

    base_model.trainable = False
    model.summary()
    return model

In [7]:
base_model = hub.KerasLayer(inception_V3, input_shape=input_shape,trainable=False)
model = build_model(base_model, input_shape, num_classes)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 332, 332, 3)]     0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 2048)              21802784  
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               204900    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303   

In [10]:
#Train the top layers of the model
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=240,
    decay_rate=0.4)

earlystop_callback = tf.keras.callbacks.EarlyStopping(
  monitor='val_loss', min_delta=0.001, patience=4)

model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['acc'])

model.fit(train_ds , epochs=epochs, validation_data=val_ds, batch_size=batch_size, callbacks=[earlystop_callback])

2022-11-19 21:55:44.293833: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 3626810496 exceeds 10% of free system memory.
2022-11-19 21:55:46.865810: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/20


2022-11-19 21:55:56.099711: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


43/43 [==============================] - 26s 326ms/step - loss: 0.8543 - acc: 0.6317 - val_loss: 0.7118 - val_acc: 0.6793
Epoch 2/20
43/43 [==============================] - 14s 316ms/step - loss: 0.6801 - acc: 0.7148 - val_loss: 0.6387 - val_acc: 0.7347
Epoch 3/20
43/43 [==============================] - 13s 300ms/step - loss: 0.6355 - acc: 0.7469 - val_loss: 0.5895 - val_acc: 0.7493
Epoch 4/20
43/43 [==============================] - 14s 320ms/step - loss: 0.6051 - acc: 0.7513 - val_loss: 0.5635 - val_acc: 0.7843
Epoch 5/20
43/43 [==============================] - 13s 295ms/step - loss: 0.5735 - acc: 0.7768 - val_loss: 0.5585 - val_acc: 0.7668
Epoch 6/20
43/43 [==============================] - 14s 318ms/step - loss: 0.5555 - acc: 0.7681 - val_loss: 0.5461 - val_acc: 0.7726
Epoch 7/20
43/43 [==============================] - 14s 312ms/step - loss: 0.5428 - acc: 0.7943 - val_loss: 0.5413 - val_acc: 0.7901
Epoch 8/20
43/43 [==============================] - 15s 345ms/step - loss: 0.524

In [11]:
#Train the entire model
base_model.trainable = True
model.summary()
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=240,
    decay_rate=0.2)

earlystop_callback = tf.keras.callbacks.EarlyStopping(
  monitor='val_loss', min_delta=0.001, patience=2, restore_best_weights=True)

model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
  #optimizer=tf.keras.optimizers.SGD(learning_rate=lr_schedule,momentum=0.9),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics=['acc'])

epochs = 20
model.fit(train_ds , epochs=epochs, validation_data=val_ds, batch_size=batch_size, callbacks=[earlystop_callback])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 332, 332, 3)]     0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 2048)              21802784  
_________________________________________________________________
dropout (Dropout)            (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 100)               204900    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303   

In [13]:
def f1_metric(model,val_ds, num_classes):
    metric = tfa.metrics.F1Score(num_classes=num_classes, threshold=None)
    y_true = np.concatenate([y for x, y in val_ds], axis=0)
    y_pred = model.predict(val_ds)
    metric.update_state(tf.one_hot(y_true, depth=num_classes), y_pred)
    result = metric.result()
    f1_score_macro = result.numpy().mean()
    print(result.numpy())
    print(f1_score_macro)

In [14]:
f1_metric(model,val_ds, num_classes)

[0.9166667  0.63333327 0.86206895]
0.80402297


In [15]:
#get the test data and output predictions
test_data = get_np_data(test_source, dataset_source, get_labels=False)

pred = np.argmax(model.predict(test_data), axis = 1)
df_pred = pd.DataFrame({'target': pred})
df_pred.to_json("./predictions.json",indent=2)

In [48]:
df_pred = pd.DataFrame({'target': pred})

df_pred.to_json("./predictions.json",indent=2)